In [ ]:
!pip3 install spacy
!pip3 install nltk

In [ ]:

from google.colab import files
files.upload()

In [ ]:
!pip3 install kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c quora-insincere-questions-classification

In [0]:
chmod 600 /root/.kaggle/kaggle.json

In [0]:
zip=ZipFile('train.csv.zip', 'r')
zip.extractall()

In [0]:
zip=ZipFile('test.csv.zip', 'r')
zip.extractall()

In [26]:
import spacy
import nltk
nlp=spacy.load('en')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import re
from collections import Counter
from zipfile import ZipFile
import gensim
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [27]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [28]:
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0



##Preprocessing

In [29]:
from sklearn.utils import resample

sincere = train[train.target == 0]
insincere = train[train.target == 1]

train = pd.concat([resample(sincere,
                     replace = False,
                     n_samples = len(insincere)), insincere])

In [30]:
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(contractions.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return contractions[match.group(0)]
    return c_re.sub(replace, text)

In [31]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

CUSTOM_FILTERS = [lambda x: x.lower(), #lowercase
                  strip_tags, # remove html tags
                  strip_punctuation, # replace punctuation with space
                  strip_multiple_whitespaces,# remove repeating whitespaces
                  strip_non_alphanum, # remove non-alphanumeric characters
                  strip_numeric, # remove numbers
                  remove_stopwords,# remove stopwords
                  strip_short # remove words less than minsize=3 characters long
                 ]

def gensim_preprocess(docs, logging=True):
    docs = [expandContractions(doc) for doc in docs]
    docs = [preprocess_string(text, CUSTOM_FILTERS) for text in docs]
    texts_out = []
    for doc in docs:
    # https://spacy.io/usage/processing-pipelines
        doc = nlp((" ".join(doc)),  # doc = text to tokenize => creates doc
                  # disable parts of the language processing pipeline we don't need here to speed up processing
                  disable=['ner', # named entity recognition
                           'tagger', # part-of-speech tagger
                           'textcat', # document label categorizer
                          ])
        texts_out.append([tok.lemma_ for tok in doc if tok.lemma_ != '-PRON-'])
    return pd.Series(texts_out)

gensim_preprocess(train.question_text.iloc[10:15])

0            [old, old, finance, economics, phd]
1     [well, procedure, learn, web, development]
2    [sociopath, know, sociopath, think, normal]
3                   [nice, thing, customer, say]
4                      [machine, use, insulator]
dtype: object

In [32]:
%time train_corpus = gensim_preprocess(train.question_text)

CPU times: user 8min 11s, sys: 1.09 s, total: 8min 13s
Wall time: 8min 12s


## N-Gramming

In [33]:
# create ngrams
ngram_phraser = models.Phrases(train_corpus, threshold=1)
ngram = models.phrases.Phraser(ngram_phraser)
#print example
print(ngram[train_corpus[0]])

# apply model to corpus
texts = [ngram[token] for token in train_corpus]

['halfway', 'point', 'short', 'length', 'possible', 'size', 'observable', 'universe']


In [34]:
# preparing ngrams for modeling
texts = [' '.join(text) for text in texts]
train['ngrams'] = texts
train.head()

,qid,question_text,target,ngrams
857763,a80f381e7bf5a2eb7189,What is the halfway point between the shortest...,0,halfway point short length possible size obser...
79319,0f87a3371bc5c1369076,Why did Mark Zuckerberg vote for Donald Trump?,0,mark_zuckerberg vote_donald trump
471047,5c3c9746832a7ad8f500,I've got a rank of 15k in JEE Mains 2017. What...,0,get_rank jee_main well_option look nit
522474,6646d843d517c6e9356d,What don't you like about the ESRB?,0,like esrb
1058289,cf5f171bd38b76ef025b,What's important for a guy in his girl - beaut...,0,important guy_girl beauty brilliance


##Logistic Regression Baseline Model

In [35]:
# represent features as BOW
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train.ngrams)

# split into test and train sets
X_train, X_test, y_train, y_test = train_test_split(train.ngrams, train.target, test_size=0.2)

In [36]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(vectorizer.transform(X_train), y_train)

print('Logistic Regression Score: ', lr.score(vectorizer.transform(X_test), y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic Regression Score:  0.8695396609330528


In [37]:
y_ = lr.predict(vectorizer.transform(X_test))

from sklearn.metrics import classification_report
print(classification_report(y_test, y_))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87     16180
           1       0.88      0.85      0.87     16144

   micro avg       0.87      0.87      0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [38]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, y_))

,0,1
0,14392,1788
1,2429,13715


##Bernoulli Naive Bayes Model

In [39]:
from sklearn.naive_bayes import BernoulliNB

bnb = BernoulliNB()
%time bnb.fit(vectorizer.transform(X_train), y_train)

print('Naive Bayes Score: ', bnb.score(vectorizer.transform(X_test), y_test))

CPU times: user 1.19 s, sys: 0 ns, total: 1.19 s
Wall time: 1.2 s
Naive Bayes Score:  0.865796312337582


In [40]:
%time bnb_y_ = bnb.predict(vectorizer.transform(X_test))

print(classification_report(y_test, bnb_y_))

CPU times: user 304 ms, sys: 0 ns, total: 304 ms
Wall time: 305 ms
              precision    recall  f1-score   support

           0       0.88      0.85      0.86     16180
           1       0.85      0.89      0.87     16144

   micro avg       0.87      0.87      0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [41]:
pd.DataFrame(confusion_matrix(y_test, bnb_y_))

,0,1
0,13681,2499
1,1839,14305


##XG Boost

In [42]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier().fit(vectorizer.transform(X_train), y_train)

print('XGBoost Score: ', xgb_model.score(vectorizer.transform(X_test), y_test))



XGBoost Score:  0.7015530256156416


In [43]:
xgb_y_ = xgb_model.predict(vectorizer.transform(X_test))

print(classification_report(y_test, xgb_y_))

              precision    recall  f1-score   support

           0       0.64      0.93      0.76     16180
           1       0.87      0.48      0.61     16144

   micro avg       0.70      0.70      0.70     32324
   macro avg       0.75      0.70      0.69     32324
weighted avg       0.75      0.70      0.69     32324



In [44]:
pd.DataFrame(confusion_matrix(y_test, xgb_y_))

,0,1
0,14987,1193
1,8454,7690


## Ensemble Model

In [45]:
from sklearn.ensemble import VotingClassifier

#create submodels
estimators = []

model1 = lr
model2 = bnb
model3 = xgb_model


estimators.append(('logistic', model1))
estimators.append(('bernoulli', model2))
estimators.append(('xgboost', model3))


# create ensemble model
%time ensemble = VotingClassifier(estimators).fit(vectorizer.transform(X_train), y_train)
print('Ensemble Score: ', ensemble.score(vectorizer.transform(X_test), y_test))

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 27 s, sys: 52 ms, total: 27.1 s
Wall time: 27.1 s
Ensemble Score:  0.8671575300086622


In [46]:
ensemble_y_ = ensemble.predict(vectorizer.transform(X_test))

print(classification_report(y_test, ensemble_y_))


              precision    recall  f1-score   support

           0       0.85      0.89      0.87     16180
           1       0.89      0.84      0.86     16144

   micro avg       0.87      0.87      0.87     32324
   macro avg       0.87      0.87      0.87     32324
weighted avg       0.87      0.87      0.87     32324



In [47]:
pd.DataFrame(confusion_matrix(y_test, ensemble_y_))

,0,1
0,14445,1735
1,2559,13585


##Submission

In [48]:
# preprocessing/lemmatizing/stemming test data
%time test_corpus = gensim_preprocess(test.question_text)
test_texts = [ngram[token] for token in test_corpus]

test_texts = [' '.join(text) for text in test_texts]
test['ngrams'] = test_texts
test.head()

CPU times: user 18min 29s, sys: 2.79 s, total: 18min 31s
Wall time: 18min 31s


,qid,question_text,ngrams
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...,woman rude_arrogant little_bite wealth power
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...,apply_college engineer bms college_engineer wa...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...,like nurse practitioner
3,000086e4b7e1c7146103,Who are entrepreneurs?,entrepreneur
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?,education make good people nowadays


In [49]:
#ensemble on test data
ensemble.fit(vectorizer.transform(train.ngrams), train.target)
prediction = ensemble.predict(vectorizer.transform(test.ngrams))

submission = pd.DataFrame({'qid':test.qid, 'prediction':prediction})
submission.to_csv('submission.csv', index=False)
submission.head()

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


,qid,prediction
0,0000163e3ea7c7a74cd7,1
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0
